In [ ]:
ipython = get_ipython()
colab   = 'google.colab' in str(ipython)
print(ipython)

if colab:
    from google.colab import drive
    drive.mount('/content/gdrive')
    %cd '/content/gdrive/My Drive/Colab Notebooks'

!ls "./"

def MODULE_FROM_NOTEBOOK(target, source, *keys): 
    from json import load
    with open(source) as notebook:
        data = load(notebook)
        with open(target,'w') as module:
            for cell in data['cells']:
                lines = cell['source']
                if type(lines) == str:
                    lines = lines.split('\n')
                line0  = (lines or [''])[0].strip()
                haskey = lambda key: line0 == f'### {key} ###'
                if cell['cell_type'] == 'code' and any(map(haskey,keys)):
                    code = '\n'.join(lines)
                    module.write(code)
                    module.write('\n')
                     
MODULE_FROM_NOTEBOOK('./ttt_tactics.py',
                     './ttt-tactics.ipynb',
                     'HEAD','GAME','DATA','MODEL','SEARCH')

from ttt_tactics import *

!cat /proc/meminfo | grep Mem

ipynb = 'ttt-tactics-train'

# %load_ext tensorboard

# !rm -rf "./tensorboard/"


In [ ]:
minloss = 2.4
model   = AlphaZeroModel()
models  = sortedfiles(f'{DIR}/model-*.h5')
model.load(models[-1])

left = 1000
while left > 0:
    left   -= 1
    path    = f'{DIR}/data-*.tfrec'
    files   = sortedfiles(path)
    dataset = loadsamples(files, buffer_size=200*1024, batch_size=1024, seed=None)
    print('path =',path)
    print('files:',len(files))

    schedule = { 0:0.0001 }
    def onschedule(epoch, lr): 
        if epoch in schedule: return schedule[epoch]  
        else:                 return lr
    scheduler = LearningRateScheduler(onschedule)
    
    @throttle(60*60)
    def onbatch(batch, logs):
        NOTIFY(f'{str(model)}, batch={batch}, logs={logs}', title=ipynb, priority=-1)
    def onepoch(epoch, logs):
        NOTIFY(f'{str(model)}, epoch={epoch}, logs={logs}', title=ipynb, priority=0)
    def ontrain(logs):
        ok = 'OK' if logs['loss'] < minloss else 'NO'
        NOTIFY(f'{str(model)}, train={ok}, logs={logs}', title=ipynb, priority=1)
    notifier = LambdaCallback(
        on_batch_end = onbatch,
        on_epoch_end = onepoch,
        on_train_end = ontrain)
    
    history = model.fit(
        x                   = dataset,
        epochs              = 10,
        verbose             = 1,
        callbacks           = [scheduler,notifier],
        workers             = 10,
        use_multiprocessing = False)
    history = history.history

    loss = history['loss'][-1]
    if loss < minloss:
        minloss = loss
        model.save(nextfile(models[-1]))

    plt.figure()
    plt.plot(history['loss'],        label='total loss')
    plt.plot(history['policy_loss'], label='policy loss')
    plt.plot(history['value_loss'],  label='value loss')
    plt.title('training history')
    plt.xlabel('epoch')
    plt.ylabel('loss')
    plt.legend(loc="upper left")
    plt.savefig(f'{DIR}/{str(model)}.png')
    plt.show()


./ttt-tactics/model-00000012.h5 Oct 22 2020 17:41:06
path = ./ttt-tactics/data-*.tfrec
files: 100
Epoch 1/10
   2864/Unknown - 1772s 614ms/step - loss: 2.3009 - value_loss: 0.7336 - policy_loss: 1.5673 - value_lr: 1.0000e-04

In [ ]:
|